In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow.keras.layers as tfl

2022-12-06 09:24:05.538731: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras.models import Model
import tensorflow.keras.optimizers as optimizers
import tensorflow.keras.metrics as metrics

In [3]:

x_train = np.load("./cleaned_data/small/x_train.npy")
y_train = np.load("./cleaned_data/small/y_train.npy")
train_mu = np.load("./cleaned_data/small/train_mu.npy")
train_std = np.load("./cleaned_data/small/train_std.npy")
x_val = np.load("./cleaned_data/small/x_val.npy")
y_val= np.load("./cleaned_data/small/y_val.npy")


In [4]:
# y = [no sack, sack, time to sack]
# x = (M, 23, 11)
x_train.shape

(5991, 23, 11)

In [5]:
import keras.backend as K

def my_loss(y_true, y_pred):
    bce = tf.keras.losses.CategoricalCrossentropy(from_logits=False)
    mse = tf.keras.losses.MeanSquaredError()
    
    # get mse of only true positives
    true_sack_mask = y_true[:1]==1
    # checked, mse function can take in array of length zero
    return bce(y_true[:,0:-1], y_pred[:,0:-1]) + mse(y_true[true_sack_mask], y_pred[true_sack_mask])

def bce_metric(y_true, y_pred):
    return K.mean(K.binary_crossentropy(y_true[:,0:-1], y_pred[:,0:-1], from_logits=False))

def mse_metric(y_true, y_pred):
    # get mse of only true positives
    true_sack_mask = y_true[:1]==1
    return K.mean(K.square(y_true[true_sack_mask] - y_pred[true_sack_mask]), axis=-1)
    # return K.mean(K.square(y_pred[:,-1] - y_true[:,-1]), axis=-1)

# https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras
def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true[:,1] * y_pred[:,1], 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true[:,1], 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true[:,1] * y_pred[:,1], 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred[:,1], 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


In [6]:
''' 
Model

how cropping layer works - https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-cropping-layers-with-keras.md
'''

' \nModel\n\nhow cropping layer works - https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-cropping-layers-with-keras.md\n'

In [7]:
''' 
Output:
[prob of no sack, prob of sack, time till sack]

If predict no sack, doesn't matter what time is (recorded as -1 in training data)
'''

def createModel(input_shape = (23,7)):
    
    X = tfl.Input(input_shape)  # define the input to the model
    flat = tfl.Flatten(input_shape=(23, 7))(X)     # Flatten to pass into linear layers
    d1 = tfl.Dense(50, activation='relu')(flat)
    d3 = tfl.Dense(3,activation=None)(d1)
    
    # have layer (batch_size, 3). Want to take (b, [0,1]) and turn them into probabilities, and keep (b, [2]) as time
    # https://datascience.stackexchange.com/questions/86740/how-to-slice-an-input-in-keras
    intermediate = tfl.Reshape((3,1), input_shape=(3,))(d3)
    
    probs = tfl.Cropping1D(cropping=(0,1))(intermediate)
    probs = tfl.Reshape((2,), input_shape=(2,1))(probs)
    probs = tfl.Activation('softmax')(probs)
    
    time = tfl.Cropping1D(cropping=(2,0))(intermediate)
    time = tfl.Reshape((1,), input_shape=(1,1))(time)
    
    # concatenate the probabilities and predicted_time_to_sack back into one layer
    out = tfl.Concatenate(axis=-1)([probs, time])
    
    model = Model(inputs=X, outputs=out)        # create model
    
    return model
    

In [8]:
model = createModel()

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 23, 7)]      0           []                               
                                                                                                  
 flatten (Flatten)              (None, 161)          0           ['input_1[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 50)           8100        ['flatten[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 3)            153         ['dense[0][0]']                  
                                                                                              

2022-12-06 09:24:12.159302: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
LEARNING_RATE = 0.001
BETA_1 = 0.9
BETA_2 = 0.999
EPS = 1e-07

opt = optimizers.Adam(
    learning_rate=LEARNING_RATE,
    beta_1=BETA_1,
    beta_2=BETA_2,
    epsilon=EPS)

model.compile(loss = my_loss, optimizer = opt, metrics = [metrics.CategoricalAccuracy(), bce_metric, mse_metric, recall, precision])

In [10]:
# Better optimizer

lr_schedule = optimizers.schedules.ExponentialDecay(
    initial_learning_rate=LEARNING_RATE,
    decay_steps=10000,
    decay_rate=0.9)

scheduled_opt = optimizers.Adam(
    learning_rate=lr_schedule,
    beta_1=BETA_1,
    beta_2=BETA_2,
    epsilon=EPS)

In [11]:
NUM_EPOCHS = 25
history = model.fit(x_train[:,:,4:], y_train, epochs=NUM_EPOCHS)

Epoch 1/25
188/188 [==============================] - 2s 3ms/step - loss: 0.3426 - categorical_accuracy: 0.6483 - bce_metric: 0.2739 - mse_metric: 0.0723 - recall: 0.0241 - precision: 0.0107
Epoch 2/25
188/188 [==============================] - 0s 2ms/step - loss: 0.2819 - categorical_accuracy: 0.7565 - bce_metric: 0.2246 - mse_metric: 0.0562 - recall: 0.0018 - precision: 0.0053   
Epoch 3/25
188/188 [==============================] - 0s 2ms/step - loss: 0.2585 - categorical_accuracy: 0.7727 - bce_metric: 0.2137 - mse_metric: 0.0441 - recall: 0.0031 - precision: 0.0106
Epoch 4/25
188/188 [==============================] - 0s 2ms/step - loss: 0.2713 - categorical_accuracy: 0.7413 - bce_metric: 0.2040 - mse_metric: 0.0663 - recall: 0.0177 - precision: 0.0266
Epoch 5/25
188/188 [==============================] - 0s 2ms/step - loss: 0.2390 - categorical_accuracy: 0.2527 - bce_metric: 0.2017 - mse_metric: 0.0378 - recall: 0.0135 - precision: 0.0266 
Epoch 6/25
188/188 [=====================

In [12]:
train_no_sack = np.round(np.sum(y_train[:,0])/len(y_train), 3)
print(f"Percentage of training plays with no sack = {train_no_sack}")

Percentage of training plays with no sack = 0.941


In [13]:
model.predict(x_train[0:5,:,4:])

1/1 [==============================] - 0s 165ms/step


array([[ 9.9827349e-01,  1.7265019e-03, -3.5695143e+00],
       [ 9.9107802e-01,  8.9220200e-03, -2.0458045e+00],
       [ 8.4786421e-01,  1.5213583e-01, -1.6954126e+00],
       [ 9.9284792e-01,  7.1520060e-03,  4.1887766e-01],
       [ 9.6189880e-01,  3.8101166e-02,  1.3119957e+00]], dtype=float32)

In [14]:
metrics_df = pd.DataFrame(history.history)


In [15]:
metrics_df.tail()

,loss,categorical_accuracy,bce_metric,mse_metric,recall,precision
20,0.175253,0.589050,0.077819,0.096791,0.514716,0.665780
21,0.130203,0.838090,0.089156,0.040707,0.432320,0.580674
22,0.090646,0.760474,0.073508,0.016895,0.509574,0.643617
23,0.068715,0.777500,0.063071,0.005398,0.565109,0.703014
24,0.089518,0.871474,0.059918,0.029455,0.574633,0.718085


In [16]:
num_no_sack = np.round(np.sum(y_val[:,0])/len(y_val), 3)
print(f"Percentage of validation plays with no sack = {num_no_sack}")

Percentage of validation plays with no sack = 0.947


In [17]:
val_loss, cat_acc, val_bce, val_mse, val_recall, val_precision = model.evaluate(x_val[:,:,4:], y_val, verbose=2)

41/41 - 1s - loss: 0.4084 - categorical_accuracy: 0.8512 - bce_metric: 0.3028 - mse_metric: 0.0969 - recall: 0.0244 - precision: 0.0244 - 520ms/epoch - 13ms/step


In [18]:

print(f"val loss = {val_loss}")
print(f"categorical accuracy = {cat_acc}")
print(f"val_bce = {val_bce}")
print(f"val_mse = {val_mse}")
print(f"val_recall = {val_recall}")
print(f"val_precision = {val_precision}")

val loss = 0.4084392189979553
categorical accuracy = 0.8512461185455322
val_bce = 0.302818238735199
val_mse = 0.096922367811203
val_recall = 0.024390241131186485
val_precision = 0.024390241131186485


In [36]:
model.predict(x_val[:,:,4:])[:,0]

41/41 [==============================] - 0s 2ms/step


array([0.98357046, 0.99988365, 0.9999479 , ..., 0.9999821 , 0.99990845,
       0.9998735 ], dtype=float32)

In [37]:
# https://www.kdnuggets.com/2021/02/saving-loading-models-tensorflow.html

model_string = f"models/third_model/weights_epochs{NUM_EPOCHS}"
model.save_weights(model_string)